In [155]:
import requests 
import bs4
from tqdm import tqdm_notebook as tqdm

# help function
def get_content(soup):
    ## 查找所有html 元素 抓出內容
    main_container = soup.find(id='main-container')
    # 把所有文字都抓出來
    all_text = main_container.text
    # 把整個內容切割透過 "-- " 切割成2個陣列
    pre_text = all_text.split('--')[0]
    # 把每段文字 根據 '\n' 切開
    texts = pre_text.split('\n')
    # 如果你爬多篇你會發現 
    contents = texts[2:]
    # 內容
    content = ''.join(contents)
    return content

def get_total_page_number(URL):
    response = requests.get(URL, headers = my_headers)
    soup = bs4.BeautifulSoup(response.text,"html.parser")
    number = int(str(soup.select('#action-bar-container > div > div.btn-group.btn-group-paging > a:nth-child(2)')[0]).split('index')[1].split('.html')[0])
    return number


def get_this_index_data(URL):
    # 設定Header與Cookie
    my_headers = {'cookie': 'over18=1;'}
    # 發送get 請求 到 ptt 八卦版
    response = requests.get(URL, headers = my_headers)

    #  把網頁程式碼(HTML) 丟入 bs4模組分析
    soup = bs4.BeautifulSoup(response.text,"html.parser")

    ## PTT 上方4個欄位
    result = soup.find_all('div','title')
    urls = []
    titles = []
    contents = []
    for i in result:
        try:
            post_url = 'https://www.ptt.cc'+str(i).split('href="')[1].split('">')[0]
            urls.append(post_url)
            titles.append(str(i).split("]")[1].split("<")[0])
            response = requests.get(post_url, headers = my_headers)
            soup = bs4.BeautifulSoup(response.text,"html.parser")
            contents.append(get_content(soup))
        except:
            pass #(本文已被刪除)
    df = pd.DataFrame()
    df['title'] = titles
    df['url'] = urls
    df['content'] = contents
    return df

def get_some_page_ptt_data(URL,last_n_page):
    total_page_number = get_total_page_number(URL)
    URLS = [f"https://www.ptt.cc/bbs/HatePolitics/index{i}.html" for i in range(total_page_number,total_page_number-last_n_page,-1)]
    df = get_this_index_data(URL) #最新的PAGE
    for u in tqdm(URLS): #最近N個PAGE
        d = get_this_index_data(u)
        df = pd.concat([df,d]) #串起來
    return df



# 爬蟲抓取資料

In [156]:
URL = "https://www.ptt.cc/bbs/HatePolitics/index.html"
last_n_page = 30
get_some_page_ptt_data(URL,last_n_page)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10388\170785279.py:62: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for u in tqdm(URLS): #最近N個PAGE


  0%|          | 0/30 [00:00<?, ?it/s]

,title,url,content
0,甲魚現在就是死魚不怕開水燙啦...,https://www.ptt.cc/bbs/HatePolitics/M.16735373...,※ 引述《jeanvanjohn (尚市長)》之銘言：: 反正沒人敢開除她，開除剛好可以學C...
1,法官問小智論文怎麼寫的,https://www.ptt.cc/bbs/HatePolitics/M.16735376...,https://youtu.be/XZzuepUxdBU根據余正煌律師的說法 法官問小智論文...
2,賴清德總統應該上不了,https://www.ptt.cc/bbs/HatePolitics/M.16735377...,看完他的發表會整體而言還是太鄉土了，這樣在中北部是沒票的但應該可以拿到民進黨的基礎鐵票3成加...
3,很明顯側翼放棄2024 只為了奪黨內權力,https://www.ptt.cc/bbs/HatePolitics/M.16735378...,很明顯的 側翼的目的已經不是2024總統了而是把民進黨內所有異議清楚大概是想著就算民進黨輸掉...
4,無論藍綠都是會失智列車的,https://www.ptt.cc/bbs/HatePolitics/M.16735378...,昔有韓粉力挺韓國瑜現有綠粉力挺小智，痛罵堅瑜不覺得這樣只是讓中間選民看笑話嗎怎麼會覺得只顧死...
...,...,...,...
14,賴清德FB 從土地到氣候 讓家園更宜居,https://www.ptt.cc/bbs/HatePolitics/M.16734372...,1.轉錄網址︰https://tinyurl.com/44zbhwvt2.轉錄來源︰賴清德F...
15,抿一下是不是跟放進去一下下很像？,https://www.ptt.cc/bbs/HatePolitics/M.16734372...,欸，最近聽到一個人被抓到酒駕，他說只有抿一下下不算酒駕，讓我想到，這是不是說我沒有偷吃，我只...
16,王世堅再飆名言「Over my dead body」！,https://www.ptt.cc/bbs/HatePolitics/M.16734374...,二十幾分鐘的訪問，王世堅有夠兇這下應該要開除王世堅了吧他太囂張了，無視蔡英文耶！為什麼你們愛...
17,DPP絕對不能開除王世堅黨籍,https://www.ptt.cc/bbs/HatePolitics/M.16734376...,先說不是反串最近傳出新竹市黨部基層要求開除王世堅黨籍我認為 dpp 這次若真的開除王世堅黨籍...


# 保存

In [157]:
df.to_excel('ptt_post.xlsx')